In [46]:
import numpy as np
from collections import Counter    

In [47]:
# defining the entropy function used to calculate Information Gain
def entropy(y):
    y=np.array(y)
    counter=Counter(y)
    counts=counter.most_common()
    probs=[]
    for i in range(len(np.unique(y))):
        probs.append(counts[i][1]/len(y))
    probs=np.array(probs)
    
    return -(np.sum(probs*np.log2(probs)))

In [ ]:
# defining the structure of a node

class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None
        

# defining the decision tree
class DecisionTree:
    def __init__(self,min_split_samples=2,max_depth=5):
        self.min_split_samples=min_split_samples
        self.max_depth=max_depth
        self.root=None
        
    def information_gain(self,X,y,feat,thres):
        parent_entropy=entropy(y)
        left_idxs,right_idxs=self.split(X,feat,thres)
        
        n_left=len(y[left_idxs])
        n_right=len(y[right_idxs])
        n=len(y)
        
        e_left=entropy(y[left_idxs])
        e_right=entropy(y[right_idxs])
        
        child_entropy=(n_left/n)*(e_left) + (n_right/n)*(e_right)
        
        ig=parent_entropy - child_entropy
        return ig
    
    def split(self,X,feat,thres):
        left_idxs=X[:,feat] <= thres
        right_idxs=X[:,feat] > thres
        
        return left_idxs,right_idxs
    
    def best_criteria(self,X,y,feat_idxs):
        best_gain=-1
        best_thres=None
        best_feat=None
        
        for feat in feat_idxs:
            threshs=np.unique(X[:,feat])
            for thres in threshs:
                gain=self.information_gain(X,y,feat,thres)
                if(gain>=best_gain):
                    best_gain=gain
                    best_thres=thres
                    best_feat=feat
                    
        return best_feat,best_thres
    
    def grow_tree(self,X,y,depth=0):
        n_labels=len(np.unique(y))
        n_samples,n_features=X.shape
        
        if(depth>=self.max_depth or n_samples<self.min_split_samples or n_labels==1):
            leaf_value=self.most_common_label(y)
            return Node(value=leaf_value)
        
        feat_idxs=np.arange(0,n_features,1)
        
        best_feat,best_thres=self.best_criteria(X,y,feat_idxs)
        left_idxs,right_idxs=self.split(X,best_feat,best_thres)
        
        left=self.grow_tree(X[left_idxs,:],y[left_idxs],depth+1)
        right=self.grow_tree(X[right_idxs,:],y[right_idxs],depth+1)
                
        return Node(best_feat, best_thres, left, right)
    
    def most_common_label(self,y):
        unique,counts=np.unique(y,return_counts=True)
        val=0
        if(len(unique)==1):
            val=unique[0]
        else:
            counts=list(counts)
            idx=counts.index(max(counts))
            val=unique[idx]
        return val
    
    def fit(self,X,y):
        self.root=self.grow_tree(X,y)
    
    def traverse_tree(self,x,node):
        if node.is_leaf_node():
            return node.value
        
        if(x[node.feature]<=node.threshold):
            return self.traverse_tree(x,node.left)

        return self.traverse_tree(x,node.right)
    
    def predict(self,X):
        return np.array([self.traverse_tree(x,self.root) for x in X])

In [53]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = DecisionTree(max_depth=4)
clf.fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.9210526315789473
